In [1]:
# Imports required libarie
import pandas as pd
import numpy as np
import spacy
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.decomposition import NMF

In [2]:
# Load spacy 
nlp = spacy.load("en_core_web_sm")

C:\Users\gaikw\AppData\Roaming\Python\Python38\site-packages\spacy\util.py:693: UserWarning: [W095] Model 'en_core_web_sm' (3.0.0) requires spaCy >=3.0.0,<3.1.0 and is incompatible with the current version (3.0.0rc5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [3]:
sc_india_1999 = pd.read_csv('Supreme_Court_cases_India_tokens_above_1999_final.csv')
if 'Unnamed: 0' in sc_india_1999.columns:
    sc_india_1999.drop(['Unnamed: 0'], axis = 1, inplace = True)
sc_india_1999.head()

,Document Id,Tokens,Corpus
0,484625,"['littttttj', 'maintainedbeforethe', 'motor', ...",littttttj maintainedbeforethe motor accident t...
1,60864,"['littttttj', 'sethijthe', 'complaint', 'thene...",littttttj sethijthe complaint thenegotiable in...
2,1373794,"['complaint', 'forwardedby', 'magistrate', 'po...",complaint forwardedby magistrate police regist...
3,55804,"['littttttj', 'sye', 'quadri', 'jthis', 'benga...",littttttj sye quadri jthis bengal electricity ...
4,301256,"['litttttttjsethijdelay', 'condonedleave', 'gr...",litttttttjsethijdelay condonedleave grantedthe...


In [4]:
sc_india_1999[sc_india_1999.Corpus.isnull()]

,Document Id,Tokens,Corpus
11697,1331690,[],NaN
11698,1001110,[],NaN
11736,1407971,[],NaN
11737,1322423,[],NaN
11738,1699303,[],NaN
11837,642450,[],NaN
11838,1820430,[],NaN
11839,95614585,[],NaN
12121,513704,[],NaN
12291,772112,[],NaN


In [5]:
sc_india_1999.dropna(inplace=True)

In [6]:
sc_india_1999.iloc[19347]

Document Id                                             43180661
Tokens         ['banumathi', 'jleave', 'prefer', 'judgment', ...
Corpus         banumathi jleave prefer judgment date pass hig...
Name: 19382, dtype: object

In [7]:
def nmf_topics(corpus):
    
    max_df = .8
    min_df=5
    topics = 20
    features = 750000
    top_words = 100
    
    tfidf_vectorizer = TfidfVectorizer(max_df = max_df, min_df = min_df, max_features = features)
    
    tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)
    tfidf_features = tfidf_vectorizer.get_feature_names()
    
    nmf = NMF(solver='mu',max_iter=10000,n_components = topics,random_state = 42, alpha = 0.1, l1_ratio = 0.5).fit(tfidf_matrix)
    print(nmf.components_)
    
    topicdict = {}
    for idx, topic in enumerate(nmf.components_):
        topicdict[idx]=" ".join([tfidf_features[i] for i in topic.argsort()[:-top_words -1:-1]])
    return tfidf_matrix, nmf, topicdict,tfidf_features


In [8]:
tfidf_matrix, nmf_test, nmfword,feature_names = nmf_topics(sc_india_1999['Corpus'])

[[7.23470185e-027 1.01380864e-037 7.80652499e-036 ... 5.25650512e-026
  1.56381748e-025 8.06268559e-026]
 [0.00000000e+000 6.60465109e-033 0.00000000e+000 ... 0.00000000e+000
  1.07377872e-039 9.79351879e-009]
 [0.00000000e+000 1.04588766e-046 0.00000000e+000 ... 0.00000000e+000
  0.00000000e+000 1.74206101e-169]
 ...
 [0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 3.49903722e-079
  0.00000000e+000 0.00000000e+000]
 [1.56289406e-069 4.95767932e-075 3.30023946e-091 ... 5.16848689e-050
  7.11590585e-047 0.00000000e+000]
 [0.00000000e+000 0.00000000e+000 5.69037889e-050 ... 1.56435242e-053
  0.00000000e+000 2.03932029e-100]]


In [9]:
#nmf_test=pd.DataFrame(nmf_test)
#nmf_test.to_pickle('nmfmodelfull.pickle')
nmfword=pd.DataFrame(nmfword,index=[1])
nmfword.to_pickle('nmfdictfull.pickle')
nmfword=pd.read_pickle('nmfdictfull.pickle')
nmfword

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
1,respondent petition writ pass judgment applica...,pw accuse evidence witness deceased prosecutio...,land acquisition acquire possession compensati...,plaintiff property decree possession sale deed...,excise good duty tribunal manufacture assessee...,offence accuse sentence ipc criminal code impr...,arbitrator arbitration award agreement contrac...,compensation tribunal award rs accident claima...,tenant rent landlord eviction premise building...,workman labour industrial employee service wor...,candidate selection appointment college univer...,vehicle insurance company insurer policy motor...,respondent bank pay rs cheque company deposit ...,post promotion service seniority appointment t...,deceased dowry death die marriage ipc cruelty ...,bail accuse anticipatory release application t...,tax income assessee assessment sale assess dea...,election candidate nomination elect corrupt co...,detention detenu detain representation authori...,pension employee service retirement regulation...


In [10]:




def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()



In [11]:
#print topic words    
print("\nTopics in NMF model:")

print_top_words(nmf_test, feature_names,25)


Topics in NMF model:
Topic #0:
respondent petition writ pass judgment application issue matter hold authority power shall provision article party th scc proceeding civil notice division single officer record time
Topic #1:
pw accuse evidence witness deceased prosecution injury trial police incident statement house occurrence ipc record conviction sentence body murder pm pws acquittal place offence dead
Topic #2:
land acquisition acquire possession compensation notification acre value market collector award development owner area sale plot reference property deed purpose sq survey revenue plan lease
Topic #3:
plaintiff property decree possession sale deed trial execute agreement plaint title appellate party share favour partition second performance execution cpc injunction evidence judgment contract civil
Topic #4:
excise good duty tribunal manufacture assessee product central notification custom tariff exemption value commissioner sale import use export manufacturer item revenue trade

In [12]:
print(nmf_test)

NMF(alpha=0.1, l1_ratio=0.5, max_iter=10000, n_components=20, random_state=42,
    solver='mu')


In [13]:
out = nmf_test.transform(tfidf_matrix)

print(out)

[[7.33044984e-03 9.87870126e-04 1.65736997e-03 ... 2.07744331e-18
  1.74588978e-11 1.13172116e-07]
 [9.19708760e-03 1.36561468e-05 4.73612415e-19 ... 2.42849545e-11
  4.45798504e-10 4.96278125e-05]
 [1.66721255e-02 1.70355216e-05 2.11250407e-15 ... 4.84152725e-15
  6.10581449e-03 1.09560787e-10]
 ...
 [2.82686159e-02 1.30902041e-14 1.26961727e-12 ... 4.06075719e-15
  7.22290717e-14 1.70289304e-12]
 [2.13316062e-02 2.55459759e-10 5.85589001e-10 ... 3.35939674e-07
  7.59593605e-11 6.79404869e-03]
 [1.76288312e-02 7.56175707e-03 5.13229215e-13 ... 1.51069124e-20
  1.19352406e-02 9.81298041e-08]]


In [14]:
print(len(out[900]))

20


In [15]:

import operator
topics = []
topics_prob=[]
for item in out:
    max_ix, max_val = max(enumerate(item), key = operator.itemgetter(1))
    topics.append(max_ix)
    topics_prob.append(max_val)


In [16]:
sc_india_1999["topicnumber"] = pd.Series(topics, index=sc_india_1999.index)
sc_india_1999["topicstrength"] = pd.Series(topics_prob, index=sc_india_1999.index)

In [17]:
sc_india_1999

,Document Id,Tokens,Corpus,topicnumber,topicstrength
0,484625,"['littttttj', 'maintainedbeforethe', 'motor', ...",littttttj maintainedbeforethe motor accident t...,7,0.072436
1,60864,"['littttttj', 'sethijthe', 'complaint', 'thene...",littttttj sethijthe complaint thenegotiable in...,12,0.055533
2,1373794,"['complaint', 'forwardedby', 'magistrate', 'po...",complaint forwardedby magistrate police regist...,5,0.056704
3,55804,"['littttttj', 'sye', 'quadri', 'jthis', 'benga...",littttttj sye quadri jthis bengal electricity ...,12,0.014614
4,301256,"['litttttttjsethijdelay', 'condonedleave', 'gr...",litttttttjsethijdelay condonedleave grantedthe...,0,0.015418
...,...,...,...,...,...
20264,104240276,"['nagappan', 'preferred', 'judgment', 'date', ...",nagappan preferred judgment date pass high jud...,1,0.070080
20265,100919378,"['nagappan', 'prefer', 'judgment', 'date', 'pa...",nagappan prefer judgment date pass high judica...,1,0.076913
20266,160092465,"['jthe', 'special', 'assail', 'order', 'date',...",jthe special assail order date pass learn sing...,3,0.035144
20267,88547322,"['raise', 'interesting', 'interplay', 'between...",raise interesting interplay betweensection ame...,0,0.021332


In [18]:
sc_india=pd.read_csv('Supreme_Court_cases_India_all.csv')


In [19]:
sc_india_recent=sc_india.iloc[28418:53424]

In [20]:




sc_india_recent.reset_index(inplace=True)



In [21]:
sc_india_recent.rename(columns={'Case Title':'Case_Title'},inplace=True)
sc_india_recent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25005 entries, 0 to 25004
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   index        25005 non-null  int64 
 1   Unnamed: 0   25005 non-null  int64 
 2   Case_Title   25005 non-null  object
 3   Year         25005 non-null  int64 
 4   Author       19861 non-null  object
 5   Judges       23602 non-null  object
 6   Document Id  25005 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 1.3+ MB


C:\Users\gaikw\anaconda3\lib\site-packages\pandas\core\frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [22]:
sc_india_recent

,index,Unnamed: 0,Case_Title,Year,Author,Judges,Document Id
0,28418,28418,Suresh Chand Jain vs State Of Madhya Pradesh &...,2001,NaN,"K.T.Thomas, R.P.Sethi",1373794
1,28419,28419,West Bengal State Electricity ... vs Patel Eng...,2001,S S Quadri,"S.S.M.Quadri, S.N.Phukan",55804
2,28420,28420,Ragu Thilak D. John vs S.Rayappan & Others on ...,2001,Sethi,"K.T. Thomas, R.P. Sethi.",301256
3,28421,28421,Karnataka State Road Transport ... vs B.S. Hul...,2001,NaN,"B.N. Kirpal, Mrs. Ruma Pal",342526
4,28422,28422,State Of Gujarat And Ors. vs Pratamsingh Narsi...,2001,NaN,"G Pattanaik, B Agrawal",315034
...,...,...,...,...,...,...,...
25000,53418,53418,Anand Kumar Tiwari vs High Court Of Madhya Pra...,2021,L. Nageswara Rao,"L. Nageswara Rao, Aniruddha Bose",105056923
25001,53419,53419,Lachhmi Narain Singh (D) Thr. ... vs Sarjug Si...,2021,Hrishikesh Roy,"Sanjay Kishan Kaul, Hrishikesh Roy",185528426
25002,53420,53420,Hemraj Ratnakar Salian vs Hdfc Bank Ltd. on 17...,2021,S. Abdul Nazeer,"S. Abdul Nazeer, Krishna Murari",172174360
25003,53421,53421,The State Of Uttar Pradesh vs Uttam Singh on 3...,2021,Sanjay Kishan Kaul,"Sanjay Kishan Kaul, Hrishikesh Roy",65485129


In [23]:
sc_india_1999['year']=pd.Series(sc_india_recent.Year)
sc_india_1999['Case_Title']=pd.Series(sc_india_recent.Case_Title)
sc_india_1999['Judges']=pd.Series(sc_india_recent.Judges)
sc_india_1999['Author']=pd.Series(sc_india_recent.Author)


In [24]:

#sc_india_1999.drop(['(year,)'],inplace=True,axis=1)


In [25]:
sc_india_1999

,Document Id,Tokens,Corpus,topicnumber,topicstrength,year,Case_Title,Judges,Author
0,484625,"['littttttj', 'maintainedbeforethe', 'motor', ...",littttttj maintainedbeforethe motor accident t...,7,0.072436,2001,Suresh Chand Jain vs State Of Madhya Pradesh &...,"K.T.Thomas, R.P.Sethi",NaN
1,60864,"['littttttj', 'sethijthe', 'complaint', 'thene...",littttttj sethijthe complaint thenegotiable in...,12,0.055533,2001,West Bengal State Electricity ... vs Patel Eng...,"S.S.M.Quadri, S.N.Phukan",S S Quadri
2,1373794,"['complaint', 'forwardedby', 'magistrate', 'po...",complaint forwardedby magistrate police regist...,5,0.056704,2001,Ragu Thilak D. John vs S.Rayappan & Others on ...,"K.T. Thomas, R.P. Sethi.",Sethi
3,55804,"['littttttj', 'sye', 'quadri', 'jthis', 'benga...",littttttj sye quadri jthis bengal electricity ...,12,0.014614,2001,Karnataka State Road Transport ... vs B.S. Hul...,"B.N. Kirpal, Mrs. Ruma Pal",NaN
4,301256,"['litttttttjsethijdelay', 'condonedleave', 'gr...",litttttttjsethijdelay condonedleave grantedthe...,0,0.015418,2001,State Of Gujarat And Ors. vs Pratamsingh Narsi...,"G Pattanaik, B Agrawal",NaN
...,...,...,...,...,...,...,...,...,...
20264,104240276,"['nagappan', 'preferred', 'judgment', 'date', ...",nagappan preferred judgment date pass high jud...,1,0.070080,2016,Raghavendra Swamy Mutt vs Uttaradi Mutt on 30 ...,"Dipak Misra, Shiva Kirti Singh",NaN
20265,100919378,"['nagappan', 'prefer', 'judgment', 'date', 'pa...",nagappan prefer judgment date pass high judica...,1,0.076913,2016,Siddharth Chaturvedi vs Securities And Exchang...,"Kurian Joseph, Rohinton Fali Nariman",........................J.
20266,160092465,"['jthe', 'special', 'assail', 'order', 'date',...",jthe special assail order date pass learn sing...,3,0.035144,2016,State Of Kerala vs P.B.Sourabhan & Ors on 4 Ma...,"Ranjan Gogoi, Prafulla C. Pant",R Gogoi
20267,88547322,"['raise', 'interesting', 'interplay', 'between...",raise interesting interplay betweensection ame...,0,0.021332,2016,"Hardei vs State Of U.P on 30 March, 2016","Dipak Misra, Shiva Kirti Singh",S K Singh


In [26]:
year_df = sc_india_1999[['year','topicnumber']]
year_df['count']=1
year_df = year_df.groupby(['year', 'topicnumber']).count().reset_index()

<ipython-input-26-f8879e249881>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  year_df['count']=1


In [27]:
year_df

,year,topicnumber,count
0,2001,0,180
1,2001,1,98
2,2001,2,63
3,2001,3,111
4,2001,4,68
...,...,...,...
313,2016,13,3
314,2016,14,5
315,2016,16,11
316,2016,17,5


In [28]:
year_df.groupby('year')['count'].sum()

year
2001    1164
2002    1341
2003    1369
2004    1213
2005     941
2006    1229
2007    1561
2008    2811
2009    2207
2010    1380
2011    1270
2012     847
2013    1066
2014     833
2015     811
2016     191
Name: count, dtype: int64

In [29]:
year_df

,year,topicnumber,count
0,2001,0,180
1,2001,1,98
2,2001,2,63
3,2001,3,111
4,2001,4,68
...,...,...,...
313,2016,13,3
314,2016,14,5
315,2016,16,11
316,2016,17,5


In [30]:
year_df = year_df.pivot("topicnumber", "year", "count")

In [31]:
year_df


year,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
topicnumber,,,,,,,,,,,,,,,,
0,180.0,196.0,212.0,213.0,164.0,213.0,266.0,720.0,551.0,326.0,283.0,213.0,248.0,144.0,153.0,54.0
1,98.0,254.0,205.0,157.0,94.0,121.0,170.0,284.0,301.0,154.0,178.0,129.0,158.0,118.0,108.0,17.0
2,63.0,67.0,101.0,73.0,45.0,52.0,86.0,136.0,112.0,106.0,107.0,43.0,60.0,57.0,46.0,22.0
3,111.0,77.0,85.0,70.0,83.0,113.0,118.0,201.0,130.0,82.0,53.0,45.0,76.0,58.0,70.0,9.0
4,68.0,125.0,163.0,109.0,87.0,74.0,109.0,128.0,52.0,54.0,60.0,23.0,15.0,17.0,56.0,4.0
5,88.0,72.0,63.0,73.0,66.0,66.0,93.0,199.0,193.0,109.0,108.0,83.0,97.0,73.0,68.0,7.0
6,30.0,26.0,54.0,25.0,19.0,47.0,53.0,80.0,55.0,37.0,39.0,22.0,19.0,36.0,24.0,4.0
7,20.0,21.0,17.0,16.0,10.0,12.0,18.0,79.0,70.0,34.0,47.0,20.0,35.0,31.0,29.0,5.0
8,69.0,97.0,45.0,42.0,39.0,28.0,37.0,79.0,61.0,36.0,24.0,17.0,11.0,23.0,7.0,5.0


In [32]:
year_df.loc[0,:]

year
2001    180.0
2002    196.0
2003    212.0
2004    213.0
2005    164.0
2006    213.0
2007    266.0
2008    720.0
2009    551.0
2010    326.0
2011    283.0
2012    213.0
2013    248.0
2014    144.0
2015    153.0
2016     54.0
Name: 0, dtype: float64

In [33]:
sc_india_1999.to_pickle('sc_india_1999.pickle')
year_df.to_pickle('year_df.pickle')

In [34]:
sc_india_1999.tail()

,Document Id,Tokens,Corpus,topicnumber,topicstrength,year,Case_Title,Judges,Author
20264,104240276,"['nagappan', 'preferred', 'judgment', 'date', ...",nagappan preferred judgment date pass high jud...,1,0.070080,2016,Raghavendra Swamy Mutt vs Uttaradi Mutt on 30 ...,"Dipak Misra, Shiva Kirti Singh",NaN
20265,100919378,"['nagappan', 'prefer', 'judgment', 'date', 'pa...",nagappan prefer judgment date pass high judica...,1,0.076913,2016,Siddharth Chaturvedi vs Securities And Exchang...,"Kurian Joseph, Rohinton Fali Nariman",........................J.
20266,160092465,"['jthe', 'special', 'assail', 'order', 'date',...",jthe special assail order date pass learn sing...,3,0.035144,2016,State Of Kerala vs P.B.Sourabhan & Ors on 4 Ma...,"Ranjan Gogoi, Prafulla C. Pant",R Gogoi
20267,88547322,"['raise', 'interesting', 'interplay', 'between...",raise interesting interplay betweensection ame...,0,0.021332,2016,"Hardei vs State Of U.P on 30 March, 2016","Dipak Misra, Shiva Kirti Singh",S K Singh
20268,34010735,"['gogoi', 'jleave', 'grantedthe', 'short', 'ar...",gogoi jleave grantedthe short arise iswhether ...,0,0.017629,2016,D Sudhakar vs State Of A P And Ors on 28 March...,"Madan B. Lokur, S.A. Bobde",NaN


In [35]:
#make a cosine similarity matrix from the tfidf matrix
from sklearn.metrics.pairwise import cosine_similarity
cosinesim = 1- cosine_similarity(tfidf_matrix)
cosinesim

array([[ 1.11022302e-16,  9.60583040e-01,  9.58980802e-01, ...,
         9.61691508e-01,  9.47239980e-01,  9.79443111e-01],
       [ 9.60583040e-01, -2.22044605e-16,  8.77163038e-01, ...,
         9.66326895e-01,  9.49345385e-01,  9.78370078e-01],
       [ 9.58980802e-01,  8.77163038e-01, -2.22044605e-16, ...,
         9.53759943e-01,  9.60078254e-01,  6.94167322e-01],
       ...,
       [ 9.61691508e-01,  9.66326895e-01,  9.53759943e-01, ...,
        -4.44089210e-16,  9.45979457e-01,  9.65427624e-01],
       [ 9.47239980e-01,  9.49345385e-01,  9.60078254e-01, ...,
         9.45979457e-01, -2.22044605e-16,  9.45448642e-01],
       [ 9.79443111e-01,  9.78370078e-01,  6.94167322e-01, ...,
         9.65427624e-01,  9.45448642e-01, -4.44089210e-16]])

In [36]:
Caseyearlist = []
for ix, row in sc_india_1999.iterrows():
    Case_Title = row.Case_Title
    year = str(row.year)
    
    Caseyearlist.append(Case_Title +"______" +year)
    

Caseyearlist

['Suresh Chand Jain vs State Of Madhya Pradesh & Another on 10 January, 2001______2001',
 'West Bengal State Electricity ... vs Patel Engineering Co. Ltd. & Ors on 15 January, 2001______2001',
 'Ragu Thilak D. John vs S.Rayappan & Others on 23 January, 2001______2001',
 'Karnataka State Road Transport ... vs B.S. Hullikatti on 22 January, 2001______2001',
 'State Of Gujarat And Ors. vs Pratamsingh Narsinh Parmar on 31 January, 2001______2001',
 'M/S Associated Cement Companies ... vs Commissioner Of Customs on 25 January, 2001______2001',
 'Shamnsaheb M.Multtani vs State Of Karnataka on 24 January, 2001______2001',
 'Dr. Vijay Laxmi Sadho vs Jagdish on 5 January, 2001______2001',
 'Gujarat Agricultural University vs Rathod Labhu Bechar & Ors on 18 January, 2001______2001',
 'Union Of India vs Elphinstone Spinning & Weaving ... on 10 January, 2001______2001',
 'V.S. Achuthanandan vs P.J. Francis & Anr on 31 January, 2001______2001',
 'The Assistant Commissioner Of ... vs Thanthi Trust E

In [37]:
cosinesimilarity = pd.DataFrame(data = cosinesim, index = Caseyearlist,  columns = Caseyearlist)

In [38]:
cosinesimilarity.loc['Tek Chand vs Dile Ram on 24 January, 2001______2001','Vividh Kamgar Sabha vs Kalyani Steels Ltd. & Anr on 9 January, 2001______2001']

0.9287901677457887

In [39]:
from sklearn.cluster import KMeans
num_clusters = 4
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)
clusters = km.labels_.tolist()

In [40]:
print(clusters)

[3, 3, 3, 3, 3, 3, 0, 3, 2, 3, 0, 3, 3, 3, 3, 3, 0, 3, 3, 0, 3, 0, 0, 0, 0, 3, 0, 2, 3, 3, 2, 0, 3, 3, 0, 0, 3, 1, 3, 3, 3, 3, 1, 3, 1, 3, 0, 0, 2, 3, 3, 3, 3, 3, 3, 3, 2, 3, 0, 3, 0, 2, 3, 3, 3, 0, 0, 3, 0, 3, 0, 0, 0, 2, 1, 2, 0, 0, 3, 3, 3, 3, 3, 3, 0, 3, 1, 3, 3, 3, 3, 1, 3, 3, 0, 3, 1, 3, 3, 1, 3, 0, 0, 3, 3, 0, 3, 2, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 0, 2, 3, 3, 3, 0, 3, 1, 2, 2, 0, 3, 3, 3, 3, 2, 3, 0, 2, 0, 2, 0, 3, 3, 3, 0, 3, 3, 3, 3, 0, 0, 3, 3, 3, 3, 0, 3, 0, 0, 1, 0, 0, 0, 0, 0, 0, 3, 3, 0, 3, 3, 2, 0, 3, 2, 3, 3, 3, 0, 3, 3, 2, 3, 0, 0, 0, 3, 3, 3, 0, 3, 3, 3, 3, 2, 3, 3, 3, 0, 0, 3, 3, 2, 3, 3, 3, 2, 3, 3, 2, 2, 0, 1, 0, 3, 0, 0, 0, 0, 3, 3, 1, 3, 3, 3, 0, 2, 3, 0, 3, 3, 3, 2, 0, 3, 3, 0, 3, 3, 3, 1, 3, 3, 3, 0, 3, 3, 0, 0, 0, 3, 2, 3, 3, 0, 3, 0, 3, 0, 3, 0, 0, 3, 3, 3, 3, 3, 3, 2, 3, 0, 3, 3, 2, 3, 2, 1, 3, 0, 2, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 2, 0, 2, 3, 3, 0, 3, 3, 0, 3, 3, 3, 3, 3, 3, 2, 3, 0, 0, 3, 3, 3, 3, 3, 2, 0, 3, 3, 1, 0, 2, 

In [41]:

clustcount = pd.Series(clusters)
print(clustcount)
#clustcount.value_counts
sc_india_1999['clusters_4'] = pd.Series(clusters, index = sc_india_1999.index)

0        3
1        3
2        3
3        3
4        3
        ..
20229    2
20230    2
20231    3
20232    3
20233    3
Length: 20234, dtype: int64


In [42]:
from sklearn.cluster import KMeans
num_clusters = 6
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)
clusters = km.labels_.tolist()

In [43]:

clustcount = pd.Series(clusters)
print(clustcount)
#clustcount.value_counts
sc_india_1999['clusters_6'] = pd.Series(clusters, index = sc_india_1999.index)

0        2
1        3
2        3
3        2
4        2
        ..
20229    4
20230    4
20231    5
20232    2
20233    3
Length: 20234, dtype: int64


In [44]:
sc_india_1999

,Document Id,Tokens,Corpus,topicnumber,topicstrength,year,Case_Title,Judges,Author,clusters_4,clusters_6
0,484625,"['littttttj', 'maintainedbeforethe', 'motor', ...",littttttj maintainedbeforethe motor accident t...,7,0.072436,2001,Suresh Chand Jain vs State Of Madhya Pradesh &...,"K.T.Thomas, R.P.Sethi",NaN,3,2
1,60864,"['littttttj', 'sethijthe', 'complaint', 'thene...",littttttj sethijthe complaint thenegotiable in...,12,0.055533,2001,West Bengal State Electricity ... vs Patel Eng...,"S.S.M.Quadri, S.N.Phukan",S S Quadri,3,3
2,1373794,"['complaint', 'forwardedby', 'magistrate', 'po...",complaint forwardedby magistrate police regist...,5,0.056704,2001,Ragu Thilak D. John vs S.Rayappan & Others on ...,"K.T. Thomas, R.P. Sethi.",Sethi,3,3
3,55804,"['littttttj', 'sye', 'quadri', 'jthis', 'benga...",littttttj sye quadri jthis bengal electricity ...,12,0.014614,2001,Karnataka State Road Transport ... vs B.S. Hul...,"B.N. Kirpal, Mrs. Ruma Pal",NaN,3,2
4,301256,"['litttttttjsethijdelay', 'condonedleave', 'gr...",litttttttjsethijdelay condonedleave grantedthe...,0,0.015418,2001,State Of Gujarat And Ors. vs Pratamsingh Narsi...,"G Pattanaik, B Agrawal",NaN,3,2
...,...,...,...,...,...,...,...,...,...,...,...
20264,104240276,"['nagappan', 'preferred', 'judgment', 'date', ...",nagappan preferred judgment date pass high jud...,1,0.070080,2016,Raghavendra Swamy Mutt vs Uttaradi Mutt on 30 ...,"Dipak Misra, Shiva Kirti Singh",NaN,2,4
20265,100919378,"['nagappan', 'prefer', 'judgment', 'date', 'pa...",nagappan prefer judgment date pass high judica...,1,0.076913,2016,Siddharth Chaturvedi vs Securities And Exchang...,"Kurian Joseph, Rohinton Fali Nariman",........................J.,2,4
20266,160092465,"['jthe', 'special', 'assail', 'order', 'date',...",jthe special assail order date pass learn sing...,3,0.035144,2016,State Of Kerala vs P.B.Sourabhan & Ors on 4 Ma...,"Ranjan Gogoi, Prafulla C. Pant",R Gogoi,3,5
20267,88547322,"['raise', 'interesting', 'interplay', 'between...",raise interesting interplay betweensection ame...,0,0.021332,2016,"Hardei vs State Of U.P on 30 March, 2016","Dipak Misra, Shiva Kirti Singh",S K Singh,3,2


In [47]:
likelytopics=pd.DataFrame(data=out,columns=['general','election','rape','marriage fraud','estate','employee-employer relation','robbery','Obscenity','Bank Money','Sudden death','murder','Committe administration','justice','family conflicts','Nationality','insurance','Road Safety','Linching','Land Disputes','Tax'])

In [48]:
likelytopics

,general,election,rape,marriage fraud,estate,employee-employer relation,robbery,Obscenity,Bank Money,Sudden death,murder,Committe administration,justice,family conflicts,Nationality,insurance,Road Safety,Linching,Land Disputes,Tax
0,0.007330,9.878701e-04,1.657370e-03,2.466137e-03,1.268191e-03,1.106220e-06,7.443967e-11,7.243642e-02,7.494438e-05,1.675701e-09,1.513684e-07,6.356475e-02,1.857037e-05,2.477967e-06,5.437797e-04,3.234372e-10,4.075173e-06,2.077443e-18,1.745890e-11,1.131721e-07
1,0.009197,1.365615e-05,4.736124e-19,7.228596e-09,1.935314e-04,1.918228e-02,2.934629e-10,4.437652e-17,1.319131e-08,5.990587e-08,1.890167e-08,8.764143e-09,5.553347e-02,9.802187e-04,6.902999e-04,1.276282e-09,4.117746e-09,2.428495e-11,4.457985e-10,4.962781e-05
2,0.016672,1.703552e-05,2.112504e-15,1.840302e-08,1.375864e-06,5.670418e-02,7.234332e-16,7.145259e-21,7.393329e-11,1.697825e-10,7.632300e-11,4.680744e-08,1.011386e-04,4.662491e-07,9.664082e-08,2.209968e-14,8.166660e-09,4.841527e-15,6.105814e-03,1.095608e-10
3,0.013269,3.225355e-11,3.831908e-10,5.790311e-04,5.349612e-03,2.686088e-09,4.308266e-03,2.264674e-06,3.891132e-11,1.713983e-03,1.032092e-06,2.722741e-08,1.461397e-02,2.760574e-07,8.536062e-07,8.478902e-12,4.413969e-12,3.371622e-11,4.034475e-09,1.504765e-11
4,0.015418,5.720179e-08,2.979152e-09,1.323335e-02,6.938960e-07,2.331743e-06,1.938333e-05,1.541472e-03,7.649755e-13,7.052781e-06,6.449027e-08,1.182345e-03,7.126982e-05,4.523602e-14,6.161533e-06,2.272486e-05,6.154520e-12,1.947854e-08,3.194373e-19,3.817033e-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20229,0.001732,7.008003e-02,4.018320e-14,1.031017e-08,7.366640e-08,2.146922e-04,9.483237e-14,1.530378e-08,1.134401e-14,4.520635e-15,2.296036e-07,9.128444e-03,2.569185e-06,8.835618e-04,1.901805e-09,1.366063e-04,5.135252e-03,8.047260e-17,3.042683e-11,5.940244e-13
20230,0.001822,7.691334e-02,4.529528e-08,1.716571e-07,1.196862e-09,8.548231e-03,3.277381e-21,3.716109e-10,2.318374e-08,7.460745e-14,1.710643e-10,8.848169e-05,9.692109e-04,1.962111e-13,4.435181e-09,2.273867e-04,1.843652e-14,3.403731e-08,2.135187e-12,4.162080e-10
20231,0.028269,1.309020e-14,1.269617e-12,3.514375e-02,1.665266e-09,5.566900e-07,1.187031e-07,7.719858e-08,2.114867e-10,4.032302e-10,5.688699e-06,8.469494e-12,4.041645e-06,3.529047e-07,2.440932e-07,1.654513e-06,1.784549e-12,4.060757e-15,7.222907e-14,1.702893e-12
20232,0.021332,2.554598e-10,5.855890e-10,1.151611e-05,1.058643e-02,5.571813e-03,1.383019e-09,6.055300e-03,3.822744e-06,6.446342e-09,1.339277e-06,1.060058e-05,4.407741e-03,3.576324e-05,1.344835e-04,5.714523e-11,9.308949e-05,3.359397e-07,7.595936e-11,6.794049e-03


In [49]:
sc_india_1999.to_pickle('sc_india_1999.pickle')
likelytopics.to_pickle('likelytopics.pickle')
year_df.to_pickle('year_df.pickle')